# MPPI алгоритм для балансировки перевернутого маятника

Реализация алгоритма стохастического оптимального управления Model Predictive Path Integral с использованием JAX, Python + Pybind11 + C++ и Python + Numpy. Алгоритм находит решение для задачи балансировки перевернутого маятника.

## Импорт библиотек и настройка окружения

In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time

In [2]:
project_root = Path.cwd().parent
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

In [3]:
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100
plt.style.use('seaborn-v0_8-darkgrid')

In [4]:
from controller.config import MPPIConfig
from visualization import PendulumAnimator, Plotter
from utils import ResultsCollector, MetricsCalculator

# Импорт реализаций MPPI
from mppi.numpy.mppi_numpy import MPPI_Numpy
from mppi.jax.mppi_jax import MPPI_JAX
from mppi.cpp.mppi_cpp import MPPI_CPP

ImportError: attempted relative import beyond top-level package

##  Описание алгоритма MPPI

Немного подробней можно прочитать в `README.md`

 ### Что такое задача перевернутого маятника?

 Есть тележка, которая может двигаться влево-вправо, и закрепленный на ней маятник, который нужно удерживать в вертикальном положении. Система неустойчива, потому что маятник падает при малейшем отклонении.

### Уравнения движения

$$
\ddot{x} = \frac{F + m\sin\theta (l\dot{\theta}^2 + g\cos\theta)}{M + m\sin^2\theta}
$$

$$
\ddot{\theta} = \frac{-F\cos\theta - m l \dot{\theta}^2 \cos\theta \sin\theta - (M+m)\sin\theta}{l (M + m\sin^2\theta)}
$$

### Как работает MPPI?

1. **Получаем текущее состояние** системы (положение, скорость тележки, угол, скорость маятника)
2. **Генерируем K случайных траекторий** управления на T шагов вперед
3. **Просчитываем каждую траекторию** через модель динамики
4. **Вычисляем стоимость** каждой траектории
5. **Взвешиваем траектории** по формуле:
   $$w_i = \exp\left( -\frac{1}{\lambda} (s_i - \min_j s_j) \right)$$
6. **Усредняем траектории** с учетом весов
7. **Применяем первое управление** из оптимальной последовательности
8. **Повторяем** весь процесс

## Панель управления параметрами

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

### Параметры системы (маятник + тележка)

In [ ]:
system_widgets = widgets.VBox([
    widgets.Label("ПАРАМЕТРЫ СИСТЕМЫ", style={'font_weight': 'bold', 'font_size': '14px'}),

    widgets.FloatSlider(
        value=1.0,
        min=0.5,
        max=5.0,
        step=0.1,
        description='Масса тележки (кг):',
        continuous_update=False,
        style={'description_width': 'initial'}
    ),

    widgets.FloatSlider(
        value=0.1,
        min=0.05,
        max=2.0,
        step=0.05,
        description='Масса маятника (кг):',
        continuous_update=False,
        style={'description_width': 'initial'}
    ),

    widgets.FloatSlider(
        value=1.0,
        min=0.5,
        max=2.0,
        step=0.1,
        description='Длина маятника (м):',
        continuous_update=False,
        style={'description_width': 'initial'}
    ),

    widgets.FloatSlider(
        value=0.0,
        min=-30,
        max=30,
        step=1,
        description='Начальный угол (°):',
        continuous_update=False,
        style={'description_width': 'initial'}
    ),

    widgets.FloatSlider(
        value=0.0,
        min=-2.0,
        max=2.0,
        step=0.1,
        description='Начальная позиция (м):',
        continuous_update=False,
        style={'description_width': 'initial'}
    ),
])

display(system_widgets)

### Параметры алгоритма MPPI

In [ ]:
algorithm_widgets = widgets.VBox([
    widgets.Label("⚙️ ПАРАМЕТРЫ АЛГОРИТМА MPPI", style={'font_weight': 'bold', 'font_size': '14px'}),

    widgets.IntSlider(
        value=1000,
        min=100,
        max=10000,
        step=100,
        description='Количество траекторий (K):',
        continuous_update=False,
        style={'description_width': 'initial'}
    ),

    widgets.IntSlider(
        value=30,
        min=10,
        max=100,
        step=5,
        description='Горизонт планирования (T):',
        continuous_update=False,
        style={'description_width': 'initial'}
    ),

    widgets.FloatSlider(
        value=1.0,
        min=0.1,
        max=10.0,
        step=0.1,
        description='Параметр λ:',
        continuous_update=False,
        style={'description_width': 'initial'}
    ),
])

display(algorithm_widgets)

### Выбор реализации

In [ ]:
implementation_selector = widgets.RadioButtons(
    options=[
        ('Python + NumPy', 'numpy'),
        ('JAX (GPU ускорение)', 'jax'),
        ('C++ (Pybind11)', 'cpp'),
        ('Сравнить все', 'compare')
    ],
    value='numpy',
    description='Реализация:',
    disabled=False,
    style={'description_width': 'initial'}
)

display(implementation_selector)

### Кнопки управления


In [ ]:
button_run = widgets.Button(
    description='▶️ Запустить симуляцию',
    button_style='success',
    tooltip='Запустить симуляцию с выбранными параметрами',
    icon='play'
)

button_stop = widgets.Button(
    description='⏹️ Остановить',
    button_style='danger',
    tooltip='Остановить текущую симуляцию',
    icon='stop',
    disabled=True
)

button_compare = widgets.Button(
    description='📊 Сравнить все',
    button_style='info',
    tooltip='Запустить все реализации и сравнить производительность',
    icon='bar-chart'
)

button_container = widgets.HBox([button_run, button_stop, button_compare])
display(button_container)

In [ ]:
output_tabs = widgets.Tab()

animation_output = widgets.Output()
plots_output = widgets.Output()
metrics_output = widgets.Output()
comparison_output = widgets.Output()

output_tabs.children = [animation_output, plots_output, metrics_output, comparison_output]
output_tabs.set_title(0, 'Анимация')
output_tabs.set_title(1, 'Графики')
output_tabs.set_title(2, 'Метрики')
output_tabs.set_title(3, 'Сравнение')

display(output_tabs)

 ## Функции для управления симуляцией

In [ ]:
current_controller = None
is_running = False
results_collector = ResultsCollector()

In [ ]:
def create_system_config():
    """Создает конфигурацию системы из виджетов"""
    # Получаем значения из виджетов
    children = system_widgets.children

    return SystemConfig(
        m_cart=children[1].value,      # Масса тележки
        m_pole=children[2].value,      # Масса маятника
        length=children[3].value,      # Длина маятника
        theta_0=np.radians(children[4].value),  # Начальный угол (в радианах)
        x_0=children[5].value          # Начальная позиция
    )

In [ ]:
def create_mppi_config():
    """Создает конфигурацию MPPI из виджетов"""
    children = algorithm_widgets.children

    return MPPIConfig(
        K=children[1].value,      # Количество траекторий
        T=children[2].value,      # Горизонт планирования
        lambda_=children[3].value # Параметр λ
    )

In [ ]:
def run_simulation(implementation):
    """Запускает симуляцию с выбранной реализацией"""
    global current_controller, is_running

    # Создаем конфигурации
    system_config = create_system_config()
    mppi_config = create_mppi_config()

    # Выбираем реализацию
    if implementation == 'numpy':
        controller = MPPIController(system_config, mppi_config, implementation='numpy')
    elif implementation == 'jax':
        controller = MPPIController(system_config, mppi_config, implementation='jax')
    elif implementation == 'cpp':
        controller = MPPIController(system_config, mppi_config, implementation='cpp')

In [ ]:
current_controller = controller
is_running = True
button_stop.disabled = False

## Запуск симуляции

In [ ]:
print(f"🚀 Запуск симуляции с параметрами:")
print(f"   📦 Система: тележка={system_config.m_cart}кг, маятник={system_config.m_pole}кг")
print(f"   ⚙️  MPPI: K={mppi_config.K}, T={mppi_config.T}, λ={mppi_config.lambda_}")
print(f"   💻 Реализация: {implementation}")

# Запускаем симуляцию на 500 шагов
trajectory = []
controls = []
costs = []
compute_times = []

with animation_output:
    clear_output(wait=True)
    print("⏳ Идет симуляция...")

    for step in range(500):
        if not is_running:
            break

        # Вычисляем управление
        control, cost, compute_time = controller.step()

        # Сохраняем результаты
        trajectory.append(controller.model.state.copy())
        controls.append(control)
        costs.append(cost)
        compute_times.append(compute_time)

        # Обновляем прогресс каждые 50 шагов
        if step % 50 == 0:
            print(f"   Шаг {step}/500, угол: {np.degrees(controller.model.state.theta):.1f}°")

print("Симуляция завершена!")

In [ ]:
 result_id = f"{implementation}_{len(results_collector.experiments)}"
    results_collector.add_result(
        result_id=result_id,
        experiment_name=f"experiment_{implementation}",
        implementation=implementation,
        system_config=system_config,
        mppi_config=mppi_config,
        states=trajectory,
        controls=controls,
        costs=costs,
        compute_times=compute_times,
        success=True
    )

## Визуализация

In [ ]:
def visualize_results(trajectory, controls, costs, implementation):
    """Визуализирует результаты симуляции"""

    # 1. Создаем анимацию
    with animation_output:
        clear_output(wait=True)
        print("🎬 Создание анимации...")

        animator = create_animation(
            trajectory=trajectory[:100],  # Первые 100 шагов для анимации
            controls=controls[:100],
            config=AnimationConfig(
                style=AnimationStyle.REALISTIC,
                fps=30,
                show_grid=True,
                show_trajectory=True,
                show_controls=True
            )
        )

        display(animator.show())

    # 2. Создаем графики
    with plots_output:
        clear_output(wait=True)
        print("📈 Построение графиков...")

        # Создаем дашборд с несколькими графиками
        fig, axes = plt.subplots(3, 2, figsize=(15, 12))

        # График 1: Угол маятника
        time_steps = list(range(len(trajectory)))
        angles = [np.degrees(state.theta) for state in trajectory]
        axes[0, 0].plot(time_steps, angles, 'b-', linewidth=2)
        axes[0, 0].set_title('Угол маятника от вертикали')
        axes[0, 0].set_xlabel('Шаг времени')
        axes[0, 0].set_ylabel('Угол (°)')
        axes[0, 0].grid(True, alpha=0.3)
        axes[0, 0].axhline(y=0, color='r', linestyle='--', alpha=0.5)

        # График 2: Сила управления
        axes[0, 1].plot(time_steps, controls, 'r-', linewidth=2)
        axes[0, 1].fill_between(time_steps, 0, controls, alpha=0.3, color='red')
        axes[0, 1].set_title('Сила, прикладываемая к тележке')
        axes[0, 1].set_xlabel('Шаг времени')
        axes[0, 1].set_ylabel('Сила (Н)')
        axes[0, 1].grid(True, alpha=0.3)

        # График 3: Положение тележки
        positions = [state.x for state in trajectory]
        axes[1, 0].plot(time_steps, positions, 'g-', linewidth=2)
        axes[1, 0].set_title('Положение тележки')
        axes[1, 0].set_xlabel('Шаг времени')
        axes[1, 0].set_ylabel('Положение (м)')
        axes[1, 0].grid(True, alpha=0.3)

        # График 4: Функция стоимости
        axes[1, 1].plot(time_steps, costs, 'purple', linewidth=2)
        axes[1, 1].set_title('Функция стоимости MPPI')
        axes[1, 1].set_xlabel('Шаг времени')
        axes[1, 1].set_ylabel('Стоимость')
        axes[1, 1].grid(True, alpha=0.3)

        # График 5: Фазовый портрет
        angles = [state.theta for state in trajectory]
        angular_velocities = [state.theta_dot for state in trajectory]
        axes[2, 0].plot(angles, angular_velocities, 'orange', linewidth=1, alpha=0.7)
        axes[2, 0].scatter(angles[0], angular_velocities[0], color='green', s=100, label='Начало', zorder=5)
        axes[2, 0].scatter(angles[-1], angular_velocities[-1], color='red', s=100, label='Конец', zorder=5)
        axes[2, 0].set_title('Фазовый портрет')
        axes[2, 0].set_xlabel('Угол (рад)')
        axes[2, 0].set_ylabel('Угловая скорость (рад/с)')
        axes[2, 0].grid(True, alpha=0.3)
        axes[2, 0].legend()

        # График 6: Время вычислений
        axes[2, 1].plot(time_steps, compute_times, 'brown', linewidth=2)
        axes[2, 1].set_title('Время вычисления управления')
        axes[2, 1].set_xlabel('Шаг времени')
        axes[2, 1].set_ylabel('Время (мс)')
        axes[2, 1].grid(True, alpha=0.3)

        plt.suptitle(f'Результаты симуляции - Реализация: {implementation}', fontsize=16)
        plt.tight_layout()
        plt.show()

    # 3. Показываем метрики
    with metrics_output:
        clear_output(wait=True)
        print("📊 Вычисление метрик...")

        # Вычисляем метрики
        metric_calculator = MetricCalculator()
        metrics = metric_calculator.compute_all({
            'states': trajectory,
            'controls': controls,
            'costs': costs,
            'compute_times': compute_times,
            'time_steps': time_steps,
            'success': True
        })

        # Отображаем метрики в виде таблицы
        print("\n📋 МЕТРИКИ ПРОИЗВОДИТЕЛЬНОСТИ\n" + "="*50)

        metric_descriptions = {
            'success': 'Успешность балансировки',
            'avg_compute_time': 'Среднее время вычисления (мс)',
            'max_angle': 'Максимальный угол отклонения (°)',
            'settling_time': 'Время стабилизации (шаги)',
            'control_effort': 'Энергия управления',
            'total_cost': 'Общая стоимость',
            'fps': 'Скорость обработки (FPS)'
        }

        for metric_key, description in metric_descriptions.items():
            if metric_key in metrics:
                value = metrics[metric_key]
                if metric_key == 'max_angle':
                    value = f"{np.degrees(value):.2f}" if isinstance(value, float) else value
                print(f"{description:30} : {value}")

        # Визуализация метрик
        fig, ax = plt.subplots(figsize=(10, 6))

        # Выбираем ключевые метрики для визуализации
        key_metrics = ['avg_compute_time', 'max_angle', 'settling_time', 'fps']
        metric_names = ['Время вычисления (мс)', 'Макс. угол (°)', 'Время стаб. (шаги)', 'Скорость (FPS)']
        metric_values = []

        for km in key_metrics:
            if km in metrics:
                if km == 'max_angle':
                    metric_values.append(np.degrees(metrics[km]))
                else:
                    metric_values.append(metrics[km])

        colors = plt.cm.Set3(np.linspace(0, 1, len(metric_values)))
        bars = ax.bar(metric_names, metric_values, color=colors, alpha=0.8)

        # Добавляем значения на столбцы
        for bar, value in zip(bars, metric_values):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + max(metric_values)*0.02,
                   f'{value:.2f}', ha='center', va='bottom')

        ax.set_ylabel('Значение')
        ax.set_title(f'Ключевые метрики - {implementation}')
        ax.grid(True, alpha=0.3, axis='y')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()


In [ ]:
visualize_results(trajectory, controls, costs, implementation)

In [ ]:
def run_all_implementations():
    """Запускает все реализации и сравнивает их"""
    implementations = ['numpy', 'jax', 'cpp']
    results = {}

    with comparison_output:
        clear_output(wait=True)
        print("⚖️ Запуск сравнения всех реализаций...")

        for impl in implementations:
            print(f"\n🔧 Тестирование реализации: {impl}")

            # Создаем конфигурации
            system_config = create_system_config()
            mppi_config = create_mppi_config()

            # Запускаем симуляцию
            controller = MPPIController(system_config, mppi_config, implementation=impl)

            # Запускаем на 100 шагов для сравнения
            trajectory = []
            controls = []
            costs = []
            compute_times = []

            for step in range(100):
                control, cost, compute_time = controller.step()
                trajectory.append(controller.model.state.copy())
                controls.append(control)
                costs.append(cost)
                compute_times.append(compute_time)

            # Вычисляем метрики
            metric_calculator = MetricCalculator()
            metrics = metric_calculator.compute_all({
                'states': trajectory,
                'controls': controls,
                'costs': costs,
                'compute_times': compute_times,
                'time_steps': list(range(len(trajectory))),
                'success': True
            })

            results[impl] = {
                'metrics': metrics,
                'avg_compute_time': np.mean(compute_times),
                'fps': 1000 / np.mean(compute_times) if np.mean(compute_times) > 0 else 0,
                'max_angle': np.max([abs(np.degrees(state.theta)) for state in trajectory])
            }

            print(f"   ✓ Среднее время: {np.mean(compute_times):.3f} мс")
            print(f"   ✓ FPS: {1000/np.mean(compute_times) if np.mean(compute_times)>0 else 0:.1f}")
            print(f"   ✓ Макс. угол: {np.max([abs(np.degrees(state.theta)) for state in trajectory]):.1f}°")

        # Строим график сравнения
        print("\n📊 ВИЗУАЛИЗАЦИЯ СРАВНЕНИЯ\n" + "="*50)

        fig, axes = plt.subplots(1, 3, figsize=(15, 5))

        # График 1: Время вычислений
        impl_names = list(results.keys())
        compute_times = [results[impl]['avg_compute_time'] for impl in impl_names]

        bars1 = axes[0].bar(impl_names, compute_times, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
        axes[0].set_title('Среднее время вычисления (мс)')
        axes[0].set_ylabel('Время (мс)')
        axes[0].grid(True, alpha=0.3, axis='y')

        for bar, value in zip(bars1, compute_times):
            axes[0].text(bar.get_x() + bar.get_width()/2., value + max(compute_times)*0.02,
                       f'{value:.2f}', ha='center', va='bottom')

        # График 2: FPS
        fps_values = [results[impl]['fps'] for impl in impl_names]

        bars2 = axes[1].bar(impl_names, fps_values, color=['#FFE66D', '#FF6B6B', '#4ECDC4'])
        axes[1].set_title('Скорость обработки (FPS)')
        axes[1].set_ylabel('FPS')
        axes[1].grid(True, alpha=0.3, axis='y')

        for bar, value in zip(bars2, fps_values):
            axes[1].text(bar.get_x() + bar.get_width()/2., value + max(fps_values)*0.02,
                       f'{value:.1f}', ha='center', va='bottom')

        # График 3: Максимальный угол
        angle_values = [results[impl]['max_angle'] for impl in impl_names]

        bars3 = axes[2].bar(impl_names, angle_values, color=['#95E1D3', '#F38181', '#FCE38A'])
        axes[2].set_title('Максимальный угол отклонения (°)')
        axes[2].set_ylabel('Угол (°)')
        axes[2].grid(True, alpha=0.3, axis='y')

        for bar, value in zip(bars3, angle_values):
            axes[2].text(bar.get_x() + bar.get_width()/2., value + max(angle_values)*0.02,
                       f'{value:.1f}', ha='center', va='bottom')

        plt.suptitle('Сравнение производительности реализаций MPPI', fontsize=16)
        plt.tight_layout()
        plt.show()

        # Вывод итогов
        print("\nИТОГИ СРАВНЕНИЯ")
        print("-" * 50)

        best_time_impl = min(results.items(), key=lambda x: x[1]['avg_compute_time'])[0]
        best_fps_impl = max(results.items(), key=lambda x: x[1]['fps'])[0]
        best_angle_impl = min(results.items(), key=lambda x: x[1]['max_angle'])[0]

        print(f"🔹 Самый быстрый: {best_time_impl} ({results[best_time_impl]['avg_compute_time']:.2f} мс)")
        print(f"🔹 Самая высокая скорость: {best_fps_impl} ({results[best_fps_impl]['fps']:.1f} FPS)")
        print(f"🔹 Лучшая стабильность: {best_angle_impl} (макс. угол: {results[best_angle_impl]['max_angle']:.1f}°)")

In [ ]:
def on_run_button_clicked(b):
    """Обработчик клика по кнопке 'Запустить симуляцию'"""
    implementation = implementation_selector.value

    if implementation == 'compare':
        run_all_implementations()
    else:
        run_simulation(implementation)

def on_stop_button_clicked(b):
    """Обработчик клика по кнопке 'Остановить'"""
    global is_running
    is_running = False
    button_stop.disabled = True
    print("⏹️ Симуляция остановлена")

def on_compare_button_clicked(b):
    """Обработчик клика по кнопке 'Сравнить все'"""
    run_all_implementations()

# Привязываем обработчики к кнопкам
button_run.on_click(on_run_button_clicked)
button_stop.on_click(on_stop_button_clicked)
button_compare.on_click(on_compare_button_clicked)


## Сохранение и загрузка результатов

In [ ]:
def save_experiment():
    """Сохраняет текущий эксперимент"""
    if results_collector.experiments:
        filename = f"experiment_{len(results_collector.experiments)}.json"
        results_collector.save(filename)
        print(f"✅ Эксперимент сохранен в {filename}")
    else:
        print("❌ Нет данных для сохранения")

def load_experiment():
    """Загружает эксперимент из файла"""
    import json
    from pathlib import Path

    files = list(Path('.').glob('experiment_*.json'))
    if files:
        latest = max(files, key=lambda x: x.stat().st_mtime)
        results_collector.load(latest.name)
        print(f"✅ Эксперимент загружен из {latest.name}")
        print(f"   Количество записей: {len(results_collector.experiments)}")
    else:
        print("❌ Нет сохраненных экспериментов")

In [ ]:
save_button = widgets.Button(
    description='💾 Сохранить эксперимент',
    button_style='primary',
    icon='save'
)

load_button = widgets.Button(
    description='📂 Загрузить эксперимент',
    button_style='primary',
    icon='folder'
)

def on_save_click(b):
    save_experiment()

def on_load_click(b):
    load_experiment()

In [ ]:
save_button.on_click(on_save_click)
load_button.on_click(on_load_click)

display(widgets.HBox([save_button, load_button]))